In [1]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Data Preprocessing
* Read in Data
* Get averages by day
* Combine the files with the particle sizes with the files with environmental factors

In [2]:
bme_file_location = "sofia_small/*bme280sof.csv"
sds_file_location = "sofia_small/*sds011sof.csv"

file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df_bme = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(bme_file_location)

df_sds = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(sds_file_location)


In [3]:
from pyspark.sql.functions import year, month
from pyspark.sql.functions import to_date
from pyspark.sql.functions import to_timestamp,date_format
from pyspark.sql import functions as F
from pyspark.sql.functions import count, avg
from pyspark.sql.functions import col


df_sds_transformed = df_sds.withColumn('year',year(df_sds.timestamp))\
    .withColumn('month', month(df_sds.timestamp))\
    .withColumn("day", date_format(col("timestamp"), "d"))\
    .withColumn("ts", to_date(col("timestamp")).cast("date"))

df_sds_transformed = df_sds_transformed.groupBy("ts").agg(avg("P1"), avg("P2")).orderBy(["ts"], ascending=True)

df_sds_transformed.show()

+----------+------------------+------------------+
|        ts|           avg(P1)|           avg(P2)|
+----------+------------------+------------------+
|2017-07-01|17.764459663706905| 8.341274009698298|
|2017-07-02| 9.846284524930946| 6.325375406399083|
|2017-07-03| 20.35557791635185|17.195223293020778|
|2017-07-04| 8.984114511906204| 6.868896334621589|
|2017-07-05|10.412705222705204| 7.964031059031034|
|2017-07-06| 10.85810864999049| 8.447780535930221|
|2017-07-07| 9.614079073024804| 7.430200547526521|
|2017-07-08| 12.10184730986929| 9.885236809576535|
|2017-07-09|12.441132935466957|10.319859653725107|
|2017-07-10|14.278580865387667|12.425794746989531|
|2017-07-11|16.458481004748865|13.907630592351836|
|2017-07-12|14.077904752827688|10.800456856017346|
|2017-07-13| 11.50965046888325| 8.878007956805918|
|2017-07-14| 5.461827450735781|  3.10989585931652|
|2017-07-15|10.245437171815821| 7.799760959824183|
|2017-07-16|11.484685678666041|  9.46174505220515|
|2017-07-17| 8.730244358596998|

In [4]:
df_bme_transformed = df_bme.withColumn('year',year(df_bme.timestamp))\
    .withColumn('month', month(df_bme.timestamp))\
    .withColumn("day", date_format(col("timestamp"), "d"))\
    .withColumn("ts", to_date(col("timestamp")).cast("date"))

df_bme_transformed = df_bme_transformed.groupBy("ts").agg(avg("pressure"), avg("temperature"), avg("humidity"))\
    .orderBy(["ts"], ascending=True)

df_bme_transformed.show()

+----------+-----------------+------------------+------------------+
|        ts|    avg(pressure)|  avg(temperature)|     avg(humidity)|
+----------+-----------------+------------------+------------------+
|2017-07-01|94572.18985080464| 33.33327613327619|32.792403355736745|
|2017-07-02|94441.42854684066|28.197254514672572| 44.52180304740427|
|2017-07-03|94668.76243252479| 18.25461707200767| 78.17694325226547|
|2017-07-04|95313.96683276288| 22.32803235375923|  50.4074079911003|
|2017-07-05|95440.82530922632|23.534423652694652|44.841247660928104|
|2017-07-06|95312.02019876736|25.778363851992424| 42.49701185958226|
|2017-07-07|95248.96706425186|27.469182004089852|40.482749797878675|
|2017-07-08|95059.96317789162|  25.7144688644688| 51.47889969005336|
|2017-07-09|95089.78527820377|27.075451422027033| 49.46747614048477|
|2017-07-10| 95128.1010232264|28.758966410703227|44.910974230932034|
|2017-07-11|95059.89666140139|30.580405242122854| 41.59478715493988|
|2017-07-12|94791.26359009359| 30.

In [5]:
combined_df = df_bme_transformed.join(df_sds_transformed, on=['ts'], how='left').orderBy(["ts"], ascending=True)
combined_df.show()

+----------+-----------------+------------------+------------------+------------------+------------------+
|        ts|    avg(pressure)|  avg(temperature)|     avg(humidity)|           avg(P1)|           avg(P2)|
+----------+-----------------+------------------+------------------+------------------+------------------+
|2017-07-01|94572.18985080464| 33.33327613327619|32.792403355736745|17.764459663706905| 8.341274009698298|
|2017-07-02|94441.42854684066|28.197254514672572| 44.52180304740427| 9.846284524930946| 6.325375406399083|
|2017-07-03|94668.76243252479| 18.25461707200767| 78.17694325226547| 20.35557791635185|17.195223293020778|
|2017-07-04|95313.96683276288| 22.32803235375923|  50.4074079911003| 8.984114511906204| 6.868896334621589|
|2017-07-05|95440.82530922632|23.534423652694652|44.841247660928104|10.412705222705204| 7.964031059031034|
|2017-07-06|95312.02019876736|25.778363851992424| 42.49701185958226| 10.85810864999049| 8.447780535930221|
|2017-07-07|95248.96706425186|27.4691

# NON-TimeSeries: Regression
* Used for baseline ideas of how a "regular" (non-time-series) would perform

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ['avg(pressure)', 'avg(temperature)', 'avg(humidity)'], outputCol = 'features')
features_df = vectorAssembler.transform(combined_df)
features_df = features_df.select(['features', 'avg(P2)'])
test_features_df = vectorAssembler.transform(x_test)
train_features_df = vectorAssembler.transform(x_train)
test_features_df = test_features_df.withColumnRenamed("avg(P2)","label")
train_features_df = train_features_df.withColumnRenamed("avg(P2)","label")

### Linear Regression (pressure, temperature, humidity) - Non Time-Series

In [ ]:
lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=50)
param_grid = ParamGridBuilder() \
            .addGrid(lr.regParam, [0.1, 0.3, 0.5]) \
            .addGrid(lr.elasticNetParam, [.5, .7, .9]) \
            .build()
cv = CrossValidator(estimator=lr, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)
cvModel = cv.fit(train_features_df)
besty = cvModel.bestModel
print("  ElasticNetParam:", besty._java_obj.parent().getElasticNetParam())
print("  RegParam:", besty._java_obj.parent().getRegParam())
test_predictions = besty.transform(test_features_df)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)
evaluator = RegressionEvaluator(
    metricName="r2",
    labelCol="label",
    predictionCol="prediction")
r2 = evaluator.evaluate(test_predictions)
print(r2)

### Generalized Linear Regression (pressure, temp, humidity) - Non Time-Series

In [ ]:
from pyspark.ml.regression import GeneralizedLinearRegression

glr = GeneralizedLinearRegression(featuresCol = 'features', labelCol='label', maxIter=50)
param_grid = ParamGridBuilder() \
            .addGrid(glr.family, ['gaussian', 'Gamma'])\
            .addGrid(glr.regParam, [0.1, 0.3, 0.5]) \
            .build()
cv = CrossValidator(estimator=glr, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)

cvModel = cv.fit(train_features_df)
besty = cvModel.bestModel
print("  family:", besty._java_obj.parent().getFamily())
print("  RegParam:", besty._java_obj.parent().getRegParam())
test_predictions = besty.transform(test_features_df)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)
evaluator = RegressionEvaluator(
    metricName="r2",
    labelCol="label",
    predictionCol="prediction")
r2 = evaluator.evaluate(test_predictions)
print(r2)

### FMR Regressor (pressure, temp, humidity) - Non Time-Series

In [ ]:
from pyspark.ml.regression import FMRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler

featureScaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures").fit(train_features_df)
fm = FMRegressor(featuresCol="scaledFeatures", stepSize=0.9)
pipeline = Pipeline(stages=[featureScaler, fm])
model = pipeline.fit(train_features_df)
test_predictions = model.transform(test_features_df)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

### Isotonic Regression (pressure, temp, humidity) - Non Time-Series

In [ ]:
from pyspark.ml.regression import IsotonicRegression

IR = IsotonicRegression()
param_grid = ParamGridBuilder() \
            .build()
cv = CrossValidator(estimator=IR, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)

cvModel = cv.fit(train_features_df)
besty = cvModel.bestModel
test_predictions = besty.transform(test_features_df)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

### GBT Regressor (pressure, temp, humidity) - Non Time-Series

In [ ]:
gbt = GBTRegressor(featuresCol = 'features', labelCol='label', maxIter=50)
param_grid = ParamGridBuilder() \
            .addGrid(gbt.maxDepth, [5, 10, 15]) \
            .addGrid(gbt.maxBins, [16]) \
            .build()
cv = CrossValidator(estimator=gbt, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)

cvModel = cv.fit(train_features_df)
besty = cvModel.bestModel
print("  max depth:", besty._java_obj.parent().getMaxDepth())
print("  max bins:", besty._java_obj.parent().getMaxBins())
test_predictions = besty.transform(test_features_df)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

# Time Series Prep: Window Functions

In [64]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor

In [54]:
from pyspark.sql.window import Window

def createWindowedDF(window_size, df, features, label):
    w = Window.orderBy("ts")
    
    # delete unnecessary columns
    drop_list = []
    for col in df.columns:
        if col != label and col != "ts":
            drop_list.append(col)
    
    feat_names = []
    for i in features:
        feat_names.append(i[4:-1])
        
    particle_size = label[4:-1]
    
    for f in range(0, len(feat_names)):             # for each feature
        for i in range(1, window_size+1):           # get all the lags
            df = df.withColumn(str(feat_names[f])+"_lag_"+str(i), F.lag(F.col(features[f]), i).over(w))
    
    #df = df.drop('ts')
    df = df.na.drop()        # drop all rows with na values
    
    df = df.drop(*drop_list)
    
    return df

In [55]:
def createFeatures(df, label):
    
    features = []
    for col in df.columns:
        if col != 'ts' and col != label:
            features.append(col)
    
    vectorAssembler = VectorAssembler(inputCols=features, outputCol='features')
    
    feature_df = vectorAssembler.transform(df)
    feature_df = feature_df.select('ts', 'features', label)
    feature_df = feature_df.withColumnRenamed(label,"label")

    return feature_df

In [61]:
def splitTrainTest(df):
    train = df.filter(F.col('ts').between("2017-07-01", "2019-01-01"))
    test = df.filter(F.col('ts').between("2018-10-02", "2019-08-01"))
    return train, test
    

# Lagging Features from past  X days
Change the first parameter to decide number of days, the list to change what features to get lags for, and last string should be equal to the column we want to predict.

In [66]:
# Data Prep
window_size = 3
feature_list = ["avg(humidity)", "avg(temperature)", "avg(pressure)"]
label = "avg(P1)"

df = createWindowedDF(window_size, combined_df, feature_list, label)
feature_df = createFeatures(df, "avg(P1)")
train, test = splitTrainTest(feature_df)
train.show()
test.show()

+----------+--------------------+------------------+
|        ts|            features|             label|
+----------+--------------------+------------------+
|2017-07-04|[78.1769432522654...| 8.984114511906204|
|2017-07-05|[50.4074079911003...|10.412705222705204|
|2017-07-06|[44.8412476609281...| 10.85810864999049|
|2017-07-07|[42.4970118595822...| 9.614079073024804|
|2017-07-08|[40.4827497978786...| 12.10184730986929|
|2017-07-09|[51.4788996900533...|12.441132935466957|
|2017-07-10|[49.4674761404847...|14.278580865387667|
|2017-07-11|[44.9109742309320...|16.458481004748865|
|2017-07-12|[41.5947871549398...|14.077904752827688|
|2017-07-13|[36.6744369576805...| 11.50965046888325|
|2017-07-14|[40.2070233102511...| 5.461827450735781|
|2017-07-15|[35.9493310063990...|10.245437171815821|
|2017-07-16|[48.9837747128601...|11.484685678666041|
|2017-07-17|[71.3457615017255...| 8.730244358596998|
|2017-07-18|[65.9973647036815...|11.758467153284702|
|2017-07-19|[55.4925901093460...|10.7572699633

## Linear Regression

In [65]:
lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=50)
param_grid = ParamGridBuilder() \
            .addGrid(lr.regParam, [0.1, 0.3, 0.5]) \
            .addGrid(lr.elasticNetParam, [.5, .7, .9]) \
            .build()
cv = CrossValidator(estimator=lr, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)

cvModel = cv.fit(train)
besty = cvModel.bestModel
print("  ElasticNetParam:", besty._java_obj.parent().getElasticNetParam())
print("  RegParam:", besty._java_obj.parent().getRegParam())
test_predictions = besty.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
lr_RMSE = evaluator.evaluate(test_predictions)
print(lr_RMSE)
evaluator = RegressionEvaluator(
    metricName="r2",
    labelCol="label",
    predictionCol="prediction")
r2 = evaluator.evaluate(test_predictions)
print(r2)

  ElasticNetParam: 0.9
  RegParam: 0.5
4.489945733073979
-1.3483381028421895


In [ ]:
glr = GeneralizedLinearRegression(featuresCol = 'features', labelCol='label', maxIter=50)
param_grid = ParamGridBuilder() \
            .addGrid(glr.family, ['gaussian', 'Gamma'])\
            .addGrid(glr.regParam, [0.1, 0.3, 0.5]) \
            .build()
cv = CrossValidator(estimator=glr, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)

cvModel = cv.fit(train)
besty = cvModel.bestModel
print("  family:", besty._java_obj.parent().getFamily())
print("  RegParam:", besty._java_obj.parent().getRegParam())
test_predictions = besty.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
glr_RMSE = evaluator.evaluate(test_predictions)
print(glr_RMSE)
evaluator = RegressionEvaluator(
    metricName="r2",
    labelCol="label",
    predictionCol="prediction")
r2 = evaluator.evaluate(test_predictions)
print(r2)

In [ ]:
gbt = GBTRegressor(featuresCol = 'features', labelCol='label', maxIter=50)
param_grid = ParamGridBuilder() \
            .addGrid(gbt.maxDepth, [5, 10, 15]) \
            .addGrid(gbt.maxBins, [16]) \
            .build()
cv = CrossValidator(estimator=gbt, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)

cvModel = cv.fit(train)
besty = cvModel.bestModel
print("  max depth:", besty._java_obj.parent().getMaxDepth())
print("  max bins:", besty._java_obj.parent().getMaxBins())
test_predictions = besty.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
gbt_RMSE = evaluator.evaluate(test_predictions)
print(gbt_RMSE)

In [ ]:
from pyspark.ml.regression import IsotonicRegression
IR = IsotonicRegression()
param_grid = ParamGridBuilder() \
            .build()
cv = CrossValidator(estimator=IR, estimatorParamMaps=param_grid, evaluator=RegressionEvaluator(), numFolds=5).setParallelism(8)

cvModel = cv.fit(train)
besty = cvModel.bestModel
test_predictions = besty.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="label",
    predictionCol="prediction")
ir_RMSE = evaluator.evaluate(test_predictions)
print(ir_RMSE)

In [ ]:
print("*** RMSE Values ***")
print("Linear Regression: " + str(lr_RMSE))
print("Generalized Linear Regression" + str(glr_RMSE))
print("GBT Regression: " + str(gbt_RMSE))
print("Isotonic Regression: " + str(ir_RMSE))